In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from sklearn import *

import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/dfs/scratch0/paroma/anaconda2/envs/babble/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/dfs/scratch0/paroma/anaconda2/envs/babble/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/dfs/scratch0/paroma/anaconda2/envs/babble/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all 

In [2]:
from snorkel.learning import LogisticRegression
disc_model = LogisticRegression()

In [ ]:
train_labels = np.load('/dfs/scratch0/paroma/reef/mscoco_ss.npy')

labels = np.load('/dfs/scratch0/paroma/data/mscoco/ground_val.npy')
feats_train = np.load('/dfs/scratch0/paroma/data/mscoco/feats_train.npy')
feats_val = np.load('/dfs/scratch0/paroma/data/mscoco/feats_val.npy')

In [ ]:
f1_all = []
pr_all = []
re_all = []
val_acc_all = []


lr_arr = [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
reg_arr = [1e-1,1e-2,1e-3]
n_epochs_arr = [5,10,25]

for lr in lr_arr:
    for reg in reg_arr:
        for n_epochs in n_epochs_arr:
            disc_model.train(feats_train, (train_labels+1.)/2., reg=reg, lr=lr, batch_size=400, n_epochs=n_epochs, rebalance=False, print_freq=0)
            predictions = disc_model.predictions(feats_val)
            
            val_acc_all.append(np.sum(predictions == labels)/float(np.shape(labels)[0]))
            f1_all.append(metrics.f1_score(labels, predictions))
            pr_all.append(metrics.precision_score(labels, predictions))
            re_all.append(metrics.recall_score(labels, predictions))

In [ ]:
ii,jj,kk = np.unravel_index(np.argmax(f1_all), (5,3,3))
print 'Best Lr: ', lr_arr[ii]
print 'Best Reg: ', reg_arr[jj]
print 'Best Epochs: ', n_epochs_arr[kk]
print 'F1 Score: ', max(f1_all)

In [ ]:
print 'Best Val Acc: ', np.max(val_acc_all)
print '\nBest F1: ', np.max(f1_all)
print 'Best Pr: ', pr_all[np.argmax(f1_all)]
print 'Best Re: ', re_all[np.argmax(f1_all)]